# Results exporter
Notebook to easily load a sacred experiment and create a CSV file ready to be used in Latex.

In [1]:
import sys
sys.path = ['..'] + sys.path
import pandas as pd
import numpy as np
import glob, sys, json, re
import matplotlib
import matplotlib.pyplot as plt
import scipy.stats as sts
from sacred_notebook_utils import plot_mean_ci, plot_all
from baselines.common.sacred_utils import load_runs, filter_runs
import scipy.stats as sts

In [2]:
runs = load_runs('../../../sacred_data/sacred_runs_master/')
print("Loaded runs:", len(runs.keys()))
final_runs = load_runs('../../../sacred_data/sacred_runs_final/')
print("Loaded final runs:", len(final_runs.keys()))

Loaded runs: 572
Loaded final runs: 704


In [7]:
pdisnn_runs_local = load_runs('../sacred_pdisnn/')
print("Loaded pdisnn runs:", len(pdisnn_runs_local.keys()))
# Merge the runs
total_runs = {}
for key, value in runs.items():
    total_runs[str(key) + '-base'] = value
for key, value in final_runs.items():
    total_runs[str(key) + '-final'] = value
for key, value in pdisnn_runs_local.items():
    total_runs[str(key) + '-local'] = value

Loaded pdisnn runs: 588


## Load the experiment we need

In [9]:
query = {
    'run.status': 'COMPLETED',
    'config.center': False,
    'config.gamma': 1.0,
    'config.clipping': False,
    'config.natural': False,
    'config.policy': 'nn',
    'config.iw_norm': 'none',
    'config.entropy': 'none',
    'config.env': 'rllab.mountain-car',
    'config.iw_method': 'pdis',
    'config.delta': 0.3,
    'config.bound': 'pdis-max-d2',
    'config.num_episodes': 100,
}
filtered_runs = filter_runs(query, total_runs)
print("Experiments:", len(filtered_runs.keys()))
print("Seeds:", ", ".join(map(str, [filtered_runs[key]['config']['seed'] for key in filtered_runs.keys()])))

Experiments: 25
Seeds: 109, 904, 160, 307, 746, 315, 545, 950, 481, 247, 15, 947, 8, 689, 236, 153, 100, 963, 245, 178, 42, 662, 712, 570, 10


## Extract the data we need for the CSV

In [10]:
runs = list(filtered_runs.keys())
rewards, timesteps = [], []
for run_key in runs:
    # Get the iteration number, used as index
    iteration = filtered_runs[run_key]['metrics']['Iteration']['values']
    # Get the number of timesteps 
    timestep = filtered_runs[run_key]['metrics']['TimestepsSoFar']['values']
    # Get reward
    reward = filtered_runs[run_key]['metrics']['EpRewMean']['values']
    rewards.append(pd.Series(reward, iteration))
    timesteps.append(pd.Series(timestep, iteration))
    # Plot IW std
    #stdiw_pdis = filtered_runs[run_key]['metrics']['StdIW_mean']['values']
    #stdiw_is = filtered_runs[run_key]['metrics']['IS_StdIW']['values']
    #ratio = (np.array(stdiw_is) - np.array(stdiw_pdis)) / np.abs(np.array(stdiw_pdis))
    #df.append(pd.Series(ratio, episodes))
    
    # Plot J std
    #stdiw_pdis = filtered_runs[run_key]['metrics']['J_sample_variance']['values']
    #stdiw_is = filtered_runs[run_key]['metrics']['IS_J_sample_variance']['values']    
    #ratio = (np.sqrt(np.array(stdiw_is)) - np.sqrt(np.array(stdiw_pdis))) / np.sqrt(np.abs(np.array(stdiw_pdis)))
    #df.append(pd.Series(ratio, episodes))
    
rewards = pd.concat(rewards, axis=1)
timesteps = pd.concat(timesteps, axis=1)

# Compute the mean reward with confidence interval
initial_columns = list(rewards)
reward_values = rewards.values
rewards['mean'] = np.mean(reward_values, axis=1)
rewards['std'] = np.std(reward_values, axis=1)
low, up = sts.t.interval(0.95, reward_values.shape[1]-1, loc=np.mean(reward_values, axis=1), scale=sts.sem(reward_values, axis=1))
rewards['low'] = low
rewards['up'] = up
rewards.drop(columns=initial_columns, inplace=True)

# Mean timesteps per iteration
rewards['timesteps'] = np.mean(timesteps.values, axis=1)
rewards.reset_index(inplace=True)
rewards.rename(columns={'index': 'iteration'}, inplace=True)
rewards = rewards.iloc[::5]
print(rewards.shape)
rewards.head()

(100, 6)


,iteration,mean,std,low,up,timesteps
0,1,-413.986291,2.846634,-415.185554,-412.787028,50000.00
5,6,-411.306211,4.088395,-413.028617,-409.583805,300000.00
10,11,-403.598116,13.336060,-409.216486,-397.979747,549866.80
15,16,-387.670605,45.720395,-406.932220,-368.408990,796200.52
20,21,-357.513543,82.914018,-392.444517,-322.582570,1030603.92


## Create the CSV

In [11]:
rewards.to_csv('mountaincar-pdis-pois.csv', index=False)